In [1]:
# import libraries
import pandas as pd
import warnings
warnings.simplefilter('ignore')

In [2]:
# Load the dataset
df=pd.read_csv('SMS_train.csv',encoding='unicode_escape')
df

,S. No.,Message_body,Label
0,1,Rofl. Its true to its name,Non-Spam
1,2,The guy did some bitching but I acted like i'd...,Non-Spam
2,3,"Pity, * was in mood for that. So...any other s...",Non-Spam
3,4,Will ü b going to esplanade fr home?,Non-Spam
4,5,This is the 2nd time we have tried 2 contact u...,Spam
...,...,...,...
952,953,hows my favourite person today? r u workin har...,Non-Spam
953,954,How much you got for cleaning,Non-Spam
954,955,Sorry da. I gone mad so many pending works wha...,Non-Spam
955,956,Wat time ü finish?,Non-Spam


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 957 entries, 0 to 956
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   S. No.        957 non-null    int64 
 1   Message_body  957 non-null    object
 2   Label         957 non-null    object
dtypes: int64(1), object(2)
memory usage: 22.6+ KB


In [4]:
df=df.drop('S. No.',axis=1)

**Text Cleaning**

In [5]:
# import the libraries
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
ps=PorterStemmer()

In [6]:
corpus=[]
for i in range(len(df)):
    rp=re.sub('[^a-zA-Z]'," ",df['Message_body'][i])
    rsw=rp.lower()
    rsw=rsw.split()
    rsw=[ps.stem(word) for word in rsw if word not in set(stopwords.words('english'))]
    rsw=" ".join(rsw)
    corpus.append(rsw)

**Vectorization**

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()
x=cv.fit_transform(corpus).toarray()

In [8]:
y=pd.get_dummies(df['Label'],drop_first=True)

In [9]:
# Train_Test Spilt
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=50)

***Modeling***

In [10]:
# Naive_Bayes classifier with default parametres
from sklearn.naive_bayes import MultinomialNB
model=MultinomialNB()
model.fit(x_train,y_train)

MultinomialNB()

In [11]:
# predictions
ypred_test=model.predict(x_test)
ypred_train=model.predict(x_train)

**Evaluation**

In [12]:
from sklearn.metrics import accuracy_score
print('Train accuracy:',accuracy_score(y_train,ypred_train))
print('Test accuracy:',accuracy_score(y_test,ypred_test))

Train accuracy: 0.9934640522875817
Test accuracy: 0.96875


In [13]:
from sklearn.model_selection import cross_val_score
print('cross val score:',cross_val_score(model,x,y,cv=5).mean())

cross val score: 0.956108202443281


**Prediction on new Data**

In [14]:
Email='UpgrdCentre Orange customer, you may now claim'

In [15]:
df_test=pd.DataFrame({'Message_body':Email},index=[0])

In [16]:
corpus=[]
for i in range(len(df_test)):
    rp=re.sub('[^a-zA-Z]'," ",df_test['Message_body'][i])
    rsw=rp.lower()
    rsw=rsw.split()
    rsw=[ps.stem(word) for word in rsw if word not in set(stopwords.words('english'))]
    rsw=" ".join(rsw)
    corpus.append(rsw)

In [17]:
df_test=cv.transform(corpus).toarray()

In [18]:
df_test

array([[0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [19]:
#prediction
pred=model.predict(df_test)
if pred==0:
    print('Spam')
else:
    print('Non_Spam')

Non_Spam
